In [59]:
%load_ext sql
%sql ibm_db_sa://hxx30689:1hsjnh5mh0b-kv3f@dashdb-txn-sbox-yp-lon02-15.services.eu-gb.bluemix.net:50000/BLUDB        

'Connected: hxx30689@BLUDB'

In [1]:
%matplotlib inline 
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt

external_links_sample = pd.read_csv('external_links_sample.csv')
outlet_information_sample = pd.read_csv('outlet_information_sample.csv')
portfolio_drinks_sample = pd.read_csv('portfolio_drinks_sample.csv')
#print(portfolio_drinks_sample)

df_links = pd.DataFrame(external_links_sample)
df_outletinfo = pd.DataFrame(outlet_information_sample)
df_drinks = pd.DataFrame(portfolio_drinks_sample)

df_links.head()



,id_ext_link,id_source,source,url,rating,normalized_rating,reviews_nr,normalized_reviews,last_refresh,id_outlet_info
0,7252488510069110,1-CY4GNE4KGEKEAA,Grab,1-CY4GNE4KGEKEAA,5,10,4,1,2020-06-26,5525260842163540
1,4090310212573940,https://www.foodpanda.my/restaurant/o2ic/warun...,Foodpanda,https://www.foodpanda.my/restaurant/o2ic/warun...,3.2,3,4,2,2020-06-26,3666898548445700
2,4452579013713210,https://www.foodpanda.my/restaurant/o2kr/iman-...,Foodpanda,https://www.foodpanda.my/restaurant/o2kr/iman-...,3.7,3,8,3,2020-06-26,3652562606379850
3,2670309040235150,https://www.foodpanda.my/restaurant/o3lt/aathi...,Foodpanda,https://www.foodpanda.my/restaurant/o3lt/aathi...,4,4,1,1,2020-06-26,127076524655342
4,8247131374479600,https://www.foodpanda.my/restaurant/o4vp/nasi-...,Foodpanda,https://www.foodpanda.my/restaurant/o4vp/nasi-...,5,10,1,1,2020-06-26,3644151255506910


In [61]:
#check how many different restaurants exist based on unique outlet_id. 
#1029 restaurants exists and a lot of them appear in more than 1 site/url
#urls stand for different beverages the restaurant offers
df_links['id_outlet_info'].value_counts()
df_outletinfo['id_outlet_info'].value_counts()
type([df_outletinfo['id_outlet_info']])

list

In [62]:
#check how many different sites these restaurants appear at
df_links['source'].value_counts()

Foodpanda    822
Grab         397
source         2
Name: source, dtype: int64

## Cleaning

### 1) delete rows where key names (column tittles) are entered randomly throughout the set

In [63]:
#get rid of unnecessary rows
df_links[df_links["source"]=="source"]
df_links = df_links.drop([500,1001],axis=0)

In [64]:
#get rid of unnecessary rows
df_drinks[df_drinks["price"]=="price"]
df_drinks = df_drinks.drop([500,1001,1502],axis=0)

In [65]:
#get rid of unnecessary rows
df_outletinfo[df_outletinfo["id_outlet_info"]=="id_outlet_info"]
df_outletinfo = df_outletinfo.drop([500,1001],axis=0)

In [66]:
df_links['source'].value_counts()

Foodpanda    822
Grab         397
Name: source, dtype: int64

In [67]:
#which is the region with the most restaurants
df_outletinfo['postal_code'].describe()

count      1030
unique       32
top       43200
freq        509
Name: postal_code, dtype: object

In [68]:
df_outletinfo.groupby('name').size()
#df_outletinfo['name']

name
10 Gram                             1
101 Sabah Pork Noodle               1
1THING FOOD HUB                     1
55 Noodle Restaurant                1
67 Cafe Western                     1
                                   ..
n Tea                               1
n Tea 潮茶 Bakery                     1
nine percent cafe                   1
sate 6 kupang                       1
九龙芝士焗饭 Kowloon Cheese Baked Rice    1
Length: 941, dtype: int64

### 2) Correcting series data type for values that should be integers and floats

In [69]:
#turning series into correct data types as ints and floats were entered as strings in the table
#-----------------------------------------------------------------------------------------------------------------
df_links['normalized_reviews'] = df_links['normalized_reviews'].astype(int)
df_links['normalized_rating'] = df_links['normalized_rating'].astype(int)

df_outletinfo['id_outlet_info'] = df_outletinfo['id_outlet_info'].astype(int)
df_outletinfo['id_outlet_info'] = pd.to_numeric(df_outletinfo['id_outlet_info'])

df_drinks['price'] = df_drinks['price'].astype(float)

df_drinks['id_ext_link'] = df_drinks['id_ext_link'].astype(int)
df_links['id_ext_link'] = df_links['id_ext_link'].astype(int)
df_links['id_outlet_info'] = df_links['id_outlet_info'].astype(int)

In [70]:
df_links['reviews_nr'].value_counts()

0       171
1        33
6        21
4        19
5        18
       ... 
501       1
928       1
286       1
1061      1
190       1
Name: reviews_nr, Length: 457, dtype: int64

In [71]:
#best rated site
#plt.bar(df_links['source'],df_links['normalized_rating'])
sources = pd.pivot_table(df_links, index = 'source', values='normalized_rating',aggfunc='mean')
sources

,normalized_rating
source,
Foodpanda,6.027981
Grab,5.637280


explore regional info of restaurants before you merge that table

## Merging

In [72]:
merged = df_links.set_index('id_ext_link').join(df_drinks.set_index('id_ext_link'))
merged

,id_source,source,url,rating,normalized_rating,reviews_nr,normalized_reviews,last_refresh,id_outlet_info,volume,...,description,brand,subbrand,company,price,image_url,category_level_1,category_level_2,alcohol_type,id_portfolio_drinks
id_ext_link,,,,,,,,,,,,,,,,,,,,,
2516088466154,https://www.foodpanda.my/restaurant/o7pg/nasi-...,Foodpanda,https://www.foodpanda.my/restaurant/o7pg/nasi-...,1,3,0,0,2020-06-26,1914281617524320,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15985012517005,https://www.foodpanda.my/restaurant/q0dr/apit-...,Foodpanda,https://www.foodpanda.my/restaurant/q0dr/apit-...,4.5,9,254,9,2020-06-26,2148513084768970,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22189090860135,1-CYWYBEL3EJTEEX,Grab,1-CYWYBEL3EJTEEX,3.4,1,5,1,2020-06-26,3577129198058150,unknown,...,NaN,Coca-Cola,Coca-Cola (Regular),Coca-Cola,3.0,NaN,Soft Drink,Cola,Non-Alcoholic,4.08582E+15
22189090860135,1-CYWYBEL3EJTEEX,Grab,1-CYWYBEL3EJTEEX,3.4,1,5,1,2020-06-26,3577129198058150,unknown,...,NaN,100Plus,100plus Active,"Fraser and Neave, Limited",3.0,NaN,Soft Drink,Sports,Non-Alcoholic,7.76631E+15
40562648837981,1-CY3UEJAHVNLDTT,Grab,1-CY3UEJAHVNLDTT,4.6,9,562,9,2020-06-26,8481421868629880,unknown,...,NaN,Pepsi,Pepsi,Etika Sdn Bhd,5.0,NaN,Soft Drink,Cola,Non-Alcoholic,4.40888E+15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967824506881230,1-CY4CG3DAKEV3L2,Grab,1-CY4CG3DAKEV3L2,4.6,9,186,7,2020-06-26,894249806892489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9973038146815930,1-CYTDG3NVE7JVJX,Grab,1-CYTDG3NVE7JVJX,4.7,9,585,9,2020-06-26,1345647755388140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9979094319758090,https://www.foodpanda.my/restaurant/o4fz/aien-...,Foodpanda,https://www.foodpanda.my/restaurant/o4fz/aien-...,5,10,1,1,2020-06-26,5228793813199470,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
merged[merged['id_portfolio_drinks'].isnull()]

,id_source,source,url,rating,normalized_rating,reviews_nr,normalized_reviews,last_refresh,id_outlet_info,volume,...,description,brand,subbrand,company,price,image_url,category_level_1,category_level_2,alcohol_type,id_portfolio_drinks
id_ext_link,,,,,,,,,,,,,,,,,,,,,
2516088466154,https://www.foodpanda.my/restaurant/o7pg/nasi-...,Foodpanda,https://www.foodpanda.my/restaurant/o7pg/nasi-...,1,3,0,0,2020-06-26,1914281617524320,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15985012517005,https://www.foodpanda.my/restaurant/q0dr/apit-...,Foodpanda,https://www.foodpanda.my/restaurant/q0dr/apit-...,4.5,9,254,9,2020-06-26,2148513084768970,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45711769566759,https://www.foodpanda.my/restaurant/n5yx/pan-m...,Foodpanda,https://www.foodpanda.my/restaurant/n5yx/pan-m...,4.4,8,32,5,2020-06-26,1681981866520200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53361030358664,MYDD10446,Grab,MYDD10446,4.7,9,15,3,2020-06-26,9222510632206960,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97260161508632,https://www.foodpanda.my/restaurant/q2qq/rnj-b...,Foodpanda,https://www.foodpanda.my/restaurant/q2qq/rnj-b...,4.2,6,43,5,2020-06-26,2689903697478940,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967824506881230,1-CY4CG3DAKEV3L2,Grab,1-CY4CG3DAKEV3L2,4.6,9,186,7,2020-06-26,894249806892489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9973038146815930,1-CYTDG3NVE7JVJX,Grab,1-CYTDG3NVE7JVJX,4.7,9,585,9,2020-06-26,1345647755388140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9979094319758090,https://www.foodpanda.my/restaurant/o4fz/aien-...,Foodpanda,https://www.foodpanda.my/restaurant/o4fz/aien-...,5,10,1,1,2020-06-26,5228793813199470,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From the above tables it is evident that there is a considerable amount of restaraunts that are not represented in the drink portfolio sample.
There are 916 ext_link_ids for which there is no information on what are the beverages offered in their respective sites. Thus, a significant part of the sample (also representing highly rated entries of the population) can provide no information on the beverages situation. It can be the case where this is an oversight and these ids should be included but it can as well be the case that these restaraunts do not offer beverages in their site entries. This could represent a market opportunity.

In [75]:
#Sort the merged table with dscending order of restaurant entries based on number of reviews and ratings.
merged_rev_sorted = merged.sort_values(['normalized_reviews',' normalized_rating'], ascending=[False,False])
merged_rev_sorted.head(10)
best_rest = merged_rev_sorted.loc[(merged_rev_sorted['normalized_rating'] >= 7) & (merged_rev_sorted['normalized_reviews'] >= 5)]
best_rest

,id_source,source,url,rating,normalized_rating,reviews_nr,normalized_reviews,last_refresh,id_outlet_info,volume,...,description,brand,subbrand,company,price,image_url,category_level_1,category_level_2,alcohol_type,id_portfolio_drinks
id_ext_link,,,,,,,,,,,,,,,,,,,,,
114944236176692,https://www.foodpanda.my/restaurant/n2ey/craft...,Foodpanda,https://www.foodpanda.my/restaurant/n2ey/craft...,4.5,9,514,10,2020-06-26,3144852145660520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480949552115768,https://www.foodpanda.my/restaurant/m5zp/ah-yi...,Foodpanda,https://www.foodpanda.my/restaurant/m5zp/ah-yi...,4.5,9,1575,10,2020-06-26,9453475394174520,unknown,...,NaN,Coca-Cola,Coca-Cola (Regular),Coca-Cola,3.80,NaN,Soft Drink,Cola,Non-Alcoholic,4.56588E+15
480949552115768,https://www.foodpanda.my/restaurant/m5zp/ah-yi...,Foodpanda,https://www.foodpanda.my/restaurant/m5zp/ah-yi...,4.5,9,1575,10,2020-06-26,9453475394174520,unknown,...,NaN,100Plus,100plus Active,"Fraser and Neave, Limited",3.80,NaN,Soft Drink,Sports,Non-Alcoholic,1.61808E+15
851573326274120,https://www.foodpanda.my/restaurant/q6zw/pappa...,Foodpanda,https://www.foodpanda.my/restaurant/q6zw/pappa...,4.5,9,584,10,2020-06-26,8481421868629880,unknown,...,NaN,Pepsi,Pepsi,Etika Sdn Bhd,4.72,NaN,Soft Drink,Cola,Non-Alcoholic,2.74304E+15
1258993283723120,https://www.foodpanda.my/restaurant/o1vz/teali...,Foodpanda,https://www.foodpanda.my/restaurant/o1vz/teali...,4.6,9,1248,10,2020-06-26,2548064705544490,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7514482946421220,MYDD10584,Grab,MYDD10584,4.4,7,57,5,2020-06-26,845771056020654,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8105469826690650,https://www.foodpanda.my/restaurant/o3rb/tok-a...,Foodpanda,https://www.foodpanda.my/restaurant/o3rb/tok-a...,4.3,7,30,5,2020-06-26,6785212712873140,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8987834663438100,https://www.foodpanda.my/restaurant/p9sx/seawe...,Foodpanda,https://www.foodpanda.my/restaurant/p9sx/seawe...,4.3,7,35,5,2020-06-26,2879937200206540,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
#We should drop the NaN rows since they cannot offer information on 
#what beverage the highest rated restaraunts offer. 
best_rest = best_rest[best_rest['id_portfolio_drinks'].notna()]
#best_rest.head(10)

# **Most preffered beverage brand, subrand and company. Effect of price, location and meal.**

In [102]:
#Exploring which is the most preffered beverage brand, subrand and company. Investigating the effect of price, location and meal.
most_pop_brand = best_rest.groupby('brand').size()
most_pop_brand = most_pop_brand.sort_values(ascending=False)
most_pop_brand.describe()
#most_pop_brand.groupby('brand').size()

count     25.000000
mean      28.400000
std       48.466483
min        1.000000
25%        4.000000
50%       10.000000
75%       24.000000
max      182.000000
dtype: float64

In [106]:
most_pop_brand.head(10)

brand
Coca-Cola           182
Pepsi               171
100Plus              80
Sprite               56
7Up                  42
Lipton               32
Mountain Dew         24
Goodday              16
A&W                  15
Heaven and Earth     15
dtype: int64

Above, we see that 710 of the most successfull restauraunts are offering beverages from brands Coca-Cola and Pepsi. The above table shows the top 10 brands. Specifically,
the top product from coca-cola is, unsuprisingly, the coca-cola (regular) and its direct competitor, the pepsi black as shown below.
So, companies Coca-Cola and Etika Sdn Bhd are two competitors with their two main cola products.

In [110]:
most_pop_brand = best_rest.groupby('subbrand').size()
most_pop_brand = most_pop_brand.sort_values(ascending=False).head(2)
most_pop_brand


subbrand
Coca-Cola (Regular)    172
Pepsi Black            114
dtype: int64

In [108]:
best_rest['subbrand'].value_counts(normalize=True) * 100;

In [80]:
most_pop_company = best_rest.groupby('company').size()
most_pop_company = most_pop_company.sort_values(ascending=False)
most_pop_company
best_rest['company'].value_counts(normalize=True) * 100

Etika Sdn Bhd                45.774648
Coca-Cola                    38.309859
Fraser and Neave, Limited    12.816901
Keurig Dr Pepper              2.535211
Spritzer Berhad               0.563380
Name: company, dtype: float64

In [56]:
# top_10_res = best_rest[['brand','company']].head(10)
# top_10_res

In [236]:
# top_10_res['company'].value_counts(normalize=True) * 100

It is evident, that although coca cola is the most popular brand amongst beverages, it does not reflect the success overall for the company of Coca-Cola.
Individually, the brands offered by the competitor, Etika Sdn Bhd , are less preffered but alltogether they prevail. It is obvious that the competitor offers more products in the market.


In [88]:
df_links.groupby('source').count()

,id_ext_link,id_source,url,rating,normalized_rating,reviews_nr,normalized_reviews,last_refresh,id_outlet_info
source,,,,,,,,,
Foodpanda,822,822,822,822,822,822,822,822,822
Grab,397,397,397,397,397,397,397,397,397


In [130]:
#sources.plot(kind='bar')

In [132]:
# me = df_links['id_outlet_info'].value_counts() > 1
# #print(me[me > 1].index[1])
# #np.array(me[me>1].index)
# print(me)
# df_links.to_numpy[0]

In [8]:
# outlet_information_sample = pd.read_csv('outlet_information_sample.csv')
# df_outletinfo = pd.DataFrame(outlet_information_sample)
df_outletinfo.loc[[52]]

,id_outlet_info,id_dashmote,activity_date,country,lat,lon,address,city,postal_code,street,house_number,currency,popularity,name,category,source,last_refresh,created_at
52,7024575396783492,293085906176835872375469224987835388269,2020-05-01,MY,3.080758,101.733491,"., South Cheras, 56000",Cheras,56000,.,NaN,MYR,NaN,Craft Cafe NonHalal,meal-delivery,dashmote,2020-06-26,2020-06-30


In [32]:
#df_outletinfo['address'] = df_outletinfo['address'].str[0].replace('.,', '')

#s = df_outletinfo['address'].loc[:]
#s[0]
#s[1]

s = df_outletinfo.iloc[:,6]
s[0]
s[0][0]

#throwed the content of column Address in a string which is seperated by the commas so the 1st character of '.' can be accessed separately
#only working for 1 row, 1 value x,y of the set-have to make it work for the whole column 
s = df_outletinfo.iloc[52,6]
print(s)
type(s)
f = s.split(", ")
f[1]

#splitted every entry/row of the core series by commas. every row's string can be accessed separately as shown below
x = df_outletinfo.address.str.split(',')
x[52][0]


#print a specific column for all the rows
df_outletinfo.iloc[:,[6]]
#print a specific row, all columns like a data frame with titles
#df_outletinfo.iloc[[52],[6]]
#print a specific row, all columns without titles
#df_outletinfo.iloc[52,6]

., South Cheras, 56000


,address
0,"Jalan Kemacahaya, Taman Kemacahaya, South Cher..."
1,"Kuala Lumpur, MY, B52, Kampung Sungai Serai, H..."
2,"Kuala Lumpur, MY, No. 43, Jalan Kemacahaya 8, ..."
3,"Kuala Lumpur, MY, Lot 1397 Kampung Bukit Raya,..."
4,"Kuala Lumpur, MY, Jalan kemacahaya Taman kemac..."
...,...
1027,"Cheras, MY, Jalan Cerdas, Lot 18113, Lraya Tim..."
1028,"Cheras, MY, Lot A55-A60, CMC Centre, Jalan Cer..."
1029,"Cheras, MY, 2, Ground Floor, Jalan Menara Gadi..."
1030,"Kuala Lumpur, MY, Unit L4-27, Fourth Floor, Te..."


In [151]:
df_drinks.loc[df_drinks['subbrand'] == 'Sprite', 'id_ext_link']
#df_drinks.describe()


0       2568584635368510
1       2568584635368510
51      3423806067259810
52      3423806067259810
66      5621373944464230
              ...       
1787    4624953791447510
1791    5862772080905770
1822     834523578396728
1862    4938124439811310
1871    5856042369907680
Name: id_ext_link, Length: 132, dtype: int64

In [27]:
df_links.describe()

,id_ext_link,id_source,source,url,rating,normalized_rating,reviews_nr,normalized_reviews,last_refresh,id_outlet_info
count,1221,1221,1221,1221,1221,1221,1221,1221,1221,1221
unique,1220,1220,3,1220,30,11,458,12,2,1030
top,id_ext_link,id_source,Foodpanda,url,4.3,3,0,0,6/26/2020,9.67477E+15
freq,2,2,822,2,149,210,171,171,1219,4


Based on the statistics above, there are several outlets(restaurants on sites) that do not offer beverages at all. 
F.e. Iman's Burger Shop - Jalan Kemacahaya (https://food.grab.com/my/en/restaurant/iman-s-burger-shop-jalan-kemacahaya-delivery/1-CY4GNE4KGEKEAA). It seems like an opportunity for Coca-Cola to add their Coca-Cola (Regular) subbrand, which is also the most commonly offered by vendors, to those outlets that are not currently in the sample drinks portfolio.



In [557]:
#best_rest only includes restaurants with reviews and rates higher than 5 and 7 respectively.
most_expensive = best_rest.sort_values('price', ascending=False)
most_expensive[['price','company','brand','category_level_1','category_level_2']].head(10)

,price,company,brand,category_level_1,category_level_2
id_ext_link,,,,,
8163744821070880,105.90,Etika Sdn Bhd,Pepsi,Soft Drink,Cola
6860674913100960,104.80,Etika Sdn Bhd,Pepsi,Soft Drink,Cola
1455662001760580,104.80,Etika Sdn Bhd,Pepsi,Soft Drink,Cola
7194032048282170,104.80,Etika Sdn Bhd,Pepsi,Soft Drink,Cola
9362307966777360,100.00,Etika Sdn Bhd,Pepsi,Soft Drink,Cola
886443881392481,97.50,Coca-Cola,Coca-Cola,Soft Drink,Cola
886443881392481,97.50,Coca-Cola,Coca-Cola,Soft Drink,Cola
1247183412216510,97.50,Coca-Cola,Coca-Cola,Soft Drink,Cola
886443881392481,97.50,Coca-Cola,Coca-Cola,Soft Drink,Cola


In [526]:
most_expensive = merged_rev_sorted.sort_values('price', ascending=False)
most_expensive[['price','company','brand','category_level_1','category_level_2']].head(10)

,price,company,brand,category_level_1,category_level_2
id_ext_link,,,,,
840178606206485,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
7193591876160250,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
9596928824760530,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
9596928824760530,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
9596928824760530,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
840178606206485,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
7193591876160250,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
7193591876160250,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola
840178606206485,130.00,Coca-Cola,Coca-Cola,Soft Drink,Cola


Based on the above it seems like coca cola does not exist in the top 5 rated restaurants. Instead, they exist in the top 10 most pricey. 
So, it could be an advice that coca cola makes its product available in the more popular restaurants for cheaper meals that are accompanied by pepsi. 

In [77]:
type(df_links)

pandas.core.frame.DataFrame

In [82]:
type(df_links.normalized_rating)

pandas.core.series.Series

In [218]:
# #df_links = df_links.drop([500,1001],axis=0)
# df_links['normalized_rating'] = df_links['normalized_rating'].astype(int)
# top_rated = df_links.sort_values('normalized_rating', ascending=False)
# top_rated.head(5)
# #top_rated.id_source
# # type(df_links.normalized_rating)
# # x = np.array(df_links.normalized_rating)
# # x.astype(int)
#top_rated.id_source

In [627]:
meals_all_rest = merged_rev_sorted[['id_outlet_info','normalized_rating','normalized_reviews','subbrand','brand','company','price','category_level_1','category_level_2','name','description']]
meals_all_rest = meals_all_rest.sort_values(['normalized_reviews','normalized_rating'], ascending=[False,False])
#meals_all_rest = meals_all_rest['price'].astype(int)
meals_all_rest = meals_all_rest[meals_all_rest['description'].notna()]
meals_all_rest = meals_all_rest[meals_all_rest['name'].notna()]
meals_all_rest.head(15)

,id_outlet_info,normalized_rating,normalized_reviews,subbrand,brand,company,price,category_level_1,category_level_2,name,description
id_ext_link,,,,,,,,,,,
4938124439811310,130251539557726,9,10,Minute Maid Pulpy O'Mango,Minute Maid,Coca-Cola,4.20,Juice,NaN,Minute Maid Mango,Bottle
4938124439811310,130251539557726,9,10,Minute Maid Pulpy Tropical,Minute Maid,Coca-Cola,4.20,Juice,NaN,Minute Maid Tropical,Bottle
411166743877158,3513064234451110,8,10,Minute Maid Pulpy Orange,Minute Maid,Coca-Cola,5.95,Juice,NaN,B2. Minute Maid Pulpy Orange,300ml
2587785914319300,8078194759838870,8,10,Pepsi,Pepsi,Etika Sdn Bhd,5.90,Soft Drink,Cola,Pepsi 1.5L PET (Blue),Pepsi
2587785914319300,8078194759838870,8,10,Pepsi,Pepsi,Etika Sdn Bhd,3.50,Soft Drink,Cola,Pepsi Can (Blue),Pepsi
2587785914319300,8078194759838870,8,10,Pepsi,Pepsi,Etika Sdn Bhd,62.50,Soft Drink,Cola,On the Go Pizza 3,"Ideal for 3-4 pax, 1 Large Pan / Hand-Stretche..."
2587785914319300,8078194759838870,8,10,Pepsi Black,Pepsi,Etika Sdn Bhd,5.90,Soft Drink,Cola,New Pepsi 1.5L PET (Black),New Pepsi
2587785914319300,8078194759838870,8,10,Pepsi Black,Pepsi,Etika Sdn Bhd,3.50,Soft Drink,Cola,New Pepsi Can (Black),New Pepsi
2587785914319300,8078194759838870,8,10,Pepsi Black,Pepsi,Etika Sdn Bhd,32.50,Soft Drink,Cola,On the Go Pizza 2,"Ideal for 2 pax, 1 Regular Pan / Hand-Stretche..."


In [554]:
meals.groupby('id_outlet_info').size().head(10)

id_outlet_info
11059805753436     8
13266284376453     1
38050898616066     2
47538895294817     1
64394848444731     1
66928388609296     1
84547976438654     5
87683297083283     1
88878342890673     2
102600050244320    2
dtype: int64

In [563]:
meals_best_rest = best_rest[['id_outlet_info','subbrand','brand','company','price','category_level_1','category_level_2','name','description']]
meals_best_rest = meals_best_rest.sort_values('price', ascending=False)
meals_best_rest.head(10)

,id_outlet_info,subbrand,brand,company,price,category_level_1,category_level_2,name,description
id_ext_link,,,,,,,,,
8163744821070880,8399433271673160,Pepsi Black,Pepsi,Etika Sdn Bhd,105.90,Soft Drink,Cola,Meal C,"4 pax - Ayam Bakar Oh-Sem (1/2 chicken), ¼ chi..."
6860674913100960,6496481963737280,Pepsi Black,Pepsi,Etika Sdn Bhd,104.80,Soft Drink,Cola,Set 3. 15-pc Combo,"15-pc Chicken, 2 Coleslaw (L), 2 Whipped Potat..."
1455662001760580,1291101154484440,Pepsi Black,Pepsi,Etika Sdn Bhd,104.80,Soft Drink,Cola,Set 3. 15-pc Combo,"15-pc Chicken, 2 Coleslaw (L), 2 Whipped Potat..."
7194032048282170,8308127544332450,Pepsi Black,Pepsi,Etika Sdn Bhd,104.80,Soft Drink,Cola,Set 3. 15-pc Combo,"15-pc Chicken, 2 Coleslaw (L), 2 Whipped Potat..."
9362307966777360,4666340803074160,Pepsi Black,Pepsi,Etika Sdn Bhd,100.00,Soft Drink,Cola,Set C (4pax),"Serves with ABOS 1/2 chicken, 1/4 chicken (Roa..."
886443881392481,7468963678577320,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 2. Raikan Ramadhan Sharing Combo 2,"2Pcs Ayam nyior pedas 1/4, 4 pcs ayam goreng, ..."
886443881392481,7468963678577320,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 5. Raikan Ramadhan Sharing Combo 5,"4 Pcs seoul sedap chikin, 2 pcs Kenny's quarte..."
1247183412216510,6432834257952240,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 1. Raikan Ramadhan Sharing Combo 1,"2 Pcs ayam nyior pedas 1/4, 4 Pcs Kenny's ayam..."
886443881392481,7468963678577320,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 1. Raikan Ramadhan Sharing Combo 1,"2 Pcs ayam nyior pedas 1/4, 4 Pcs Kenny's ayam..."


In [600]:
#meals_best_rest['category_level_2'] = meals_best_rest['category_level_2'].astype(int)
cola_meals = meals_best_rest[meals_best_rest["category_level_2"]=="Cola"]
cola_meals = cola_meals[cola_meals['description'].notna()]
cola_meals.head(20)
#cola_meals.groupby('description').size()

,id_outlet_info,subbrand,brand,company,price,category_level_1,category_level_2,name,description
id_ext_link,,,,,,,,,
8163744821070880,8399433271673160,Pepsi Black,Pepsi,Etika Sdn Bhd,105.90,Soft Drink,Cola,Meal C,"4 pax - Ayam Bakar Oh-Sem (1/2 chicken), ¼ chi..."
6860674913100960,6496481963737280,Pepsi Black,Pepsi,Etika Sdn Bhd,104.80,Soft Drink,Cola,Set 3. 15-pc Combo,"15-pc Chicken, 2 Coleslaw (L), 2 Whipped Potat..."
1455662001760580,1291101154484440,Pepsi Black,Pepsi,Etika Sdn Bhd,104.80,Soft Drink,Cola,Set 3. 15-pc Combo,"15-pc Chicken, 2 Coleslaw (L), 2 Whipped Potat..."
7194032048282170,8308127544332450,Pepsi Black,Pepsi,Etika Sdn Bhd,104.80,Soft Drink,Cola,Set 3. 15-pc Combo,"15-pc Chicken, 2 Coleslaw (L), 2 Whipped Potat..."
9362307966777360,4666340803074160,Pepsi Black,Pepsi,Etika Sdn Bhd,100.00,Soft Drink,Cola,Set C (4pax),"Serves with ABOS 1/2 chicken, 1/4 chicken (Roa..."
886443881392481,7468963678577320,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 2. Raikan Ramadhan Sharing Combo 2,"2Pcs Ayam nyior pedas 1/4, 4 pcs ayam goreng, ..."
886443881392481,7468963678577320,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 5. Raikan Ramadhan Sharing Combo 5,"4 Pcs seoul sedap chikin, 2 pcs Kenny's quarte..."
1247183412216510,6432834257952240,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 1. Raikan Ramadhan Sharing Combo 1,"2 Pcs ayam nyior pedas 1/4, 4 Pcs Kenny's ayam..."
886443881392481,7468963678577320,Coca-Cola (Regular),Coca-Cola,Coca-Cola,97.50,Soft Drink,Cola,RRSC 1. Raikan Ramadhan Sharing Combo 1,"2 Pcs ayam nyior pedas 1/4, 4 Pcs Kenny's ayam..."


In [629]:
meals = ['Nachos', 'beef', 'fish', 'chicken']
for meal in meals:
    cola_meals[meal] = cola_meals.astype(str).sum(axis=1).str.contains(meal)
    
#cola_meals[meal].groupby('True').size()
cola_meals[meal]

id_ext_link
8163744821070880     True
6860674913100960    False
1455662001760580    False
7194032048282170    False
9362307966777360     True
                    ...  
2970036282878420    False
2970036282878420    False
2970036282878420    False
8018676143508640    False
2970036282878420    False
Name: chicken, Length: 193, dtype: bool

In [630]:
print("Schema:\n\n",best_rest.dtypes)

Schema:

 id_source               object
source                  object
url                     object
rating                 float64
normalized_rating        int64
reviews_nr               int64
normalized_reviews       int64
last_refresh            object
id_outlet_info           int64
volume                  object
name                    object
description             object
brand                   object
subbrand                object
company                 object
price                  float64
image_url               object
category_level_1        object
category_level_2        object
alcohol_type            object
id_portfolio_drinks     object
dtype: object


In [7]:
df_outletinfo.loc

[df_outletinfo.loc[(df_outletinfo['normalized_rating'] == 7)]

id_outlet_info                                      465523422893347
id_dashmote                 118467761189075487598963389187298235612
activity_date                                            2020-05-01
country                                                          MY
lat                                                        3.084082
lon                                                      101.788392
address           Kuala Lumpur, MY, No 5, Taman Desa Pelangi Hul...
city                                                         Cheras
postal_code                                                   43100
street              No 5, Taman Desa Pelangi Hulu Langat , Malaysia
house_number                                                    NaN
currency                                                        MYR
popularity                                                      NaN
name                                     Nasi Ayam Golek Haji Yusof
category                                        

In [53]:
#df['B'].map(lambda x: re.sub(r'\W+', '', x))

df_drinks['name'].str.replace('\W', '')
df_drinks.iloc[8:30,[2]]

df_drinks.name.dtype
df_outletinfo['address'].str.replace('\W', '')

0          JalanKemacahayaTamanKemacahayaSouthCheras43100
1       KualaLumpurMYB52KampungSungaiSeraiHuluLangat43100
2       KualaLumpurMYNo43JalanKemacahaya8TamanKemacaha...
3       KualaLumpurMYLot1397KampungBukitRayaBatu10Hulu...
4        KualaLumpurMYJalankemacahayaTamankemacahaya43200
                              ...                        
1027    CherasMYJalanCerdasLot18113LrayaTimurBaratTama...
1028    CherasMYLotA55A60CMCCentreJalanCergasTamanConn...
1029    CherasMY2GroundFloorJalanMenaraGading1TamanCon...
1030    KualaLumpurMYUnitL427FourthFloorTerminalBersep...
1031    CherasMYNo2JalanJujur1BandarTunRazakkualalumpu...
Name: address, Length: 1032, dtype: object

In [127]:
# plot_data = df_drinks.groupby('brand')['price'].sum
# plot_data.plot(kind='bar')
# plt.title("brand")
# plt.ylabel("price")